# Static Paramters

In [1]:
# Directories
data_src = 'data-raw/'
data_dst = 'data-processed/'

list_to_track = ['GOLD','GBP','JPY','EUR','DAX']
time = 4

# Files
file_names = dict()
for s in list_to_track:
    file_names[s] = s+'.csv'

# Modules

In [2]:
import os # path.join
import pandas as pd # read_csv
import numpy as np # np.nan

# Load Data

In [3]:
# File Data Dictionary
file_data = dict()

# Read and Store Data
for file_key in file_names:
    # Get File Path
    file_name = file_names[file_key]
    file_path = os.path.join( data_src, file_name )
    # Read
    data = pd.read_csv( file_path, parse_dates=[0] )
    # Rename Columns
    data.columns = [ 'Date', 'Value' ]
    # Do Conversion if required
    if( file_key in [ 'GOLD', 'EUR', 'DAX' ] ):
        data['Value'] = 1 / data['Value']
    # Sort by Date
    data.sort_values( ['Date'], axis = 0, inplace=True )
    # Reset Index
    data.reset_index( drop=True, inplace=True )
    # Add Boolean Column
    # Continuous or not?!
    #data['is_continuous'] = (data.Date - data.Date.shift(1)).map( lambda x: x.days <= 1 )
    # Calculate Growth
    data[file_key] = (data['Value'] / data['Value'].shift(1)).map( lambda x: 'increase' if x > 1 else 'decrease' )
    # Filter
    #data = data[ data.is_continuous ]
    # Drop / Select
    data.drop( labels=['Value'], axis=1, inplace=True )
    
    #setting time frames
    for t in range(time):
        col_name = file_key+'_'+str(t)
        data.rename(columns={data.columns[1]: col_name}, inplace=True)
        file_data[ col_name ] = data.copy()
        data[col_name] = data[col_name].shift(1)
        
        

# Merge Data

In [6]:
df_data = {
    'inner' : pd.DataFrame(),
    'outer' : pd.DataFrame()
}
for merge_type in df_data:
    # Merge
    for key in file_data:
        if( df_data[merge_type].shape[0] == 0 ):
            df_data[merge_type] = file_data[key].copy()
        else:
            df_data[merge_type] = pd.merge( df_data[merge_type], file_data[key], on = 'Date', how = merge_type )
            
    # Drop Date
    df_data[merge_type].drop( labels = ['Date'], axis = 1, inplace = True )
    df_data[merge_type].replace(np.nan,'N/A',inplace=True)

# Store

In [5]:
for key in df_data:
    # Path
    file_name = '{}.dat'.format( key )
    file_path = os.path.join( data_dst, file_name )
    # Store
    df_data[key].to_csv( file_path, index = False )